In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,8)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs

In [3]:
import dask_cudf
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import xgboost as xgb
import gc


def get_cluster():
    cluster = LocalCUDACluster(protocol="ucx", 
                               rmm_pool_size="29GB")
    client = Client(cluster)
    return client


In [4]:
client = get_cluster()
client

2023-06-21 15:23:07,150 - distributed.comm.ucx - WARNING - A CUDA context for device 0 (b'GPU-67be321d-9012-d150-fdc9-402004163697') already exists on process ID 2839854. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-21 15:23:08,845 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
202

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: ucx://127.0.0.1:39147,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 1.48 TiB
Comm: ucx://127.0.0.1:40733,Total threads: 1
Dashboard: http://127.0.0.1:45943/status,Memory: 188.82 GiB
Nanny: ucx://127.0.0.1:40777,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq')

CPU times: user 1.07 s, sys: 685 ms, total: 1.75 s
Wall time: 1.58 s


In [6]:
VER = 186
USE = 'clicks'

In [7]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [8]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])
del users

CPU times: user 1.86 s, sys: 772 ms, total: 2.63 s
Wall time: 15.3 s


In [9]:
gc.collect()

414

In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

INFO:distributed.scheduler:Receive client connection: Client-worker-3fa73ed6-1082-11ee-9665-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:39147
INFO:distributed.scheduler:Receive client connection: Client-worker-3faa847b-1082-11ee-966d-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:39147
INFO:distributed.scheduler:Receive client connection: Client-worker-3fb04965-1082-11ee-966f-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:39147
INFO:distributed.scheduler:Receive client connection: Client-worker-3ff591c0-1082-11ee-9667-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:39147
INFO:distributed.scheduler:Receive client connection: Client-worker-3ff6f096-1082-11ee-965b-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:39147
INFO:distributed.scheduler:Receive client connection: Client-worker-3fff1a4f-108

[0]	train-map:0.16849
[1]	train-map:0.17529
[2]	train-map:0.17651
[3]	train-map:0.17744
[4]	train-map:0.18096
[5]	train-map:0.18082
[6]	train-map:0.18101
[7]	train-map:0.18109
[8]	train-map:0.18196
[9]	train-map:0.18605
[10]	train-map:0.18828
[11]	train-map:0.18831
[12]	train-map:0.18835
[13]	train-map:0.18865
[14]	train-map:0.18897
[15]	train-map:0.18954
[16]	train-map:0.18963
[17]	train-map:0.18963
[18]	train-map:0.18989
[19]	train-map:0.18989
[20]	train-map:0.19028
[21]	train-map:0.19058
[22]	train-map:0.19136
[23]	train-map:0.19151
[24]	train-map:0.19152
[25]	train-map:0.19172
[26]	train-map:0.19163
[27]	train-map:0.19211
[28]	train-map:0.19218
[29]	train-map:0.19271
[30]	train-map:0.19287
[31]	train-map:0.19300
[32]	train-map:0.19311
[33]	train-map:0.19350
[34]	train-map:0.19434
[35]	train-map:0.19512
[36]	train-map:0.19580
[37]	train-map:0.19632
[38]	train-map:0.19645
[39]	train-map:0.19670
[40]	train-map:0.19675
[41]	train-map:0.19686
[42]	train-map:0.19697
[43]	train-map:0.1970

In [12]:
duration = time()-start
duration

80.5040853023529